# The Stanford Sentiment Treebank 
The Stanford Sentiment Treebank consists of sentences from movie reviews and human annotations of their sentiment. The task is to predict the sentiment of a given sentence. We use the two-way (positive/negative) class split, and use only sentence-level labels.

In [1]:
from IPython.display import display, Markdown
with open('../../doc/env_variables_setup.md', 'r') as fh:
    content = fh.read()
display(Markdown(content))

Environment variables that need to be defined:   
`export DIR_PROJ=your_path_git_repository`  
`export PYTHONPATH=$DIR_PROJ/src`  
`export PATH_TENSORBOARD=your_path_tensorboard`  
`export PATH_DATASETS=your_path_datasets`  
`export PROJECT_ID=your_gcp_project_id`  
`export BUCKET_NAME=your_gcp_gs_bucket_name`  
`export REGION=your_region`  
`export PATH_SAVE_MODEL=your_path_to_save_model` 

- Use local Jupyter Lab 
    - you need to have the `jupyter-notebook` Anaconda python environment created [link](local_jupyter_lab_installation.md) 
    - you need to have the `jupyter-notebook` Anaconda python environment activated [link](local_jupyter_lab_installation.md) 
    - then define the environment variables above (copy and paste) 
    - you need to have the `env_multilingual_class` Anaconda python environment created [link](local_jupyter_lab_installation.md)  
    - start Jupyter Lab:  `jupyter lab` 
    - open a Jupyter Lab notebook from `notebook/` 
     - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - choose the proper Anaconda python environment:  `Python [conda env:env_multilingual_class]` [link](conda_env.md) 
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`


- Use GCP Jupyter Lab 
    - Go on GCP
    - open a Cloud Shell
    - `ssh-keygen -t rsa -b 4096 -C firstName_lastName`
    - `cp .ssh/id_rsa.pub .`
    - use Cloud Editor to edit this file `id_rsa.pub` and copy the full content
    - Go on Compute Engine -> Metadata
    - Click SSH Keys
    - Click Edit
    - Click + Add item, copy the content of `id_rsa.pub`
    - You should see firstName_lastName of the left
    - Click Save
    - you need to start a AI Platform instance 
    - open a Jupyter Lab terminal and got to `/home/gcp_user_name/`
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - then `cd proj_multilingual_text_classification/`
    - create the Anacond Python environment `conda env create -f env/environment.yml`
    - create a file `config.sh` in `/home` with the following information: 
    ```
    #!/bin/bash
    
    echo "applying some configuration ..."
    git config --global user.email user_email
    git config --global user.name user_name
    git config --global credential.helper store
        
    # Add here the enviroment variables from above below
    # [EDIT ME]
    export DIR_PROJ=your_path_git_repository
    export PYTHONPATH=$DIR_PROJ/src
  
    cd /home/gcp_user_name/
    
    conda activate env_multilingual_class

    export PS1='\[\e[91m\]\u@:\[\e[32m\]\w\[\e[0m\]$'
    ```
    - Got to AI Platform Notebook, select your instance and click "Reset".
    - Wait and reshreh you Web browser with the Notebook


## Import Packages

In [2]:
import tensorflow as tf
import tensorflow_datasets

from tensorflow.keras.utils import to_categorical

from transformers import (
    BertConfig,
    BertTokenizer,
    XLMRobertaTokenizer,
    TFBertModel,
    TFXLMRobertaModel,
    TFBertForSequenceClassification,
    glue_convert_examples_to_features,
    glue_processors
)

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

from google.cloud import storage

import math
import numpy as np
import os
import glob
import time
from datetime import timedelta
import shutil
from datetime import datetime
import pickle

## Check configuration

In [3]:
print(tf.version.GIT_VERSION, tf.version.VERSION)

v2.1.0-rc2-17-ge5bf8de410 2.1.0


In [4]:
print(tf.keras.__version__)

2.2.4-tf


In [5]:
gpus = tf.config.list_physical_devices('GPU')
if len(gpus)>0:
    for gpu in gpus:
        print('Name:', gpu.name, '  Type:', gpu.device_type)
else:
    print('No GPU available !!!!')

No GPU available !!!!


## Define Paths

In [6]:
try:
    data_dir=os.environ['PATH_DATASETS']
except KeyError:
    print('missing PATH_DATASETS')
try:   
    tensorboard_dir=os.environ['PATH_TENSORBOARD']
except KeyError:
    print('missing PATH_TENSORBOARD')
try:   
    savemodel_dir=os.environ['PATH_SAVE_MODEL']
except KeyError:
    print('missing PATH_SAVE_MODEL')

## Import local packages

In [7]:
import preprocessing.preprocessing as pp
import utils.model_metrics as mm
import utils.model_utils as mu

/Users/tarrade/anaconda-release/conda-env/env_multilingual_class/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [60]:
import importlib
importlib.reload(pp);
importlib.reload(mm);
importlib.reload(mu);

## Loading a data from Tensorflow Datasets

In [9]:
data, info = tensorflow_datasets.load(name='glue/sst2',
                                      data_dir=data_dir,
                                      with_info=True)

INFO:absl:Overwrite dataset info from restored data version.
INFO:absl:Reusing dataset glue (/Users/tarrade/tensorflow_datasets/glue/sst2/1.0.0)
INFO:absl:Constructing tf.data.Dataset for split None, from /Users/tarrade/tensorflow_datasets/glue/sst2/1.0.0


### Checking baics info from the metadata

In [10]:
info

tfds.core.DatasetInfo(
    name='glue',
    version=1.0.0,
    description='GLUE, the General Language Understanding Evaluation benchmark
(https://gluebenchmark.com/) is a collection of resources for training,
evaluating, and analyzing natural language understanding systems.

            The Stanford Sentiment Treebank consists of sentences from movie reviews and
            human annotations of their sentiment. The task is to predict the sentiment of a
            given sentence. We use the two-way (positive/negative) class split, and use only
            sentence-level labels.',
    homepage='https://nlp.stanford.edu/sentiment/index.html',
    features=FeaturesDict({
        'idx': tf.int32,
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'sentence': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=70042,
    splits={
        'test': 1821,
        'train': 67349,
        'validation': 872,
    },
    supervised_keys=None,
    citation="""@

In [11]:
pp.print_info_dataset(info)

Labels:
      ['negative', 'positive']

Number of label:
      2

Structure of the data:
      dict_keys(['sentence', 'label', 'idx'])

Number of entries:
   Train dataset: 67349
   Test dataset:  1821
   Valid dataset: 872



### Checking baics info from the metadata

In [12]:
data

{'test': <DatasetV1Adapter shapes: {idx: (), label: (), sentence: ()}, types: {idx: tf.int32, label: tf.int64, sentence: tf.string}>,
 'train': <DatasetV1Adapter shapes: {idx: (), label: (), sentence: ()}, types: {idx: tf.int32, label: tf.int64, sentence: tf.string}>,
 'validation': <DatasetV1Adapter shapes: {idx: (), label: (), sentence: ()}, types: {idx: tf.int32, label: tf.int64, sentence: tf.string}>}

In [13]:
data.keys()

dict_keys(['test', 'train', 'validation'])

In [14]:
pp.print_info_data(data['train'])

# Structure of the data:

   <DatasetV1Adapter shapes: {idx: (), label: (), sentence: ()}, types: {idx: tf.int32, label: tf.int64, sentence: tf.string}>

# Output shape of one entry:
   {'idx': TensorShape([]), 'label': TensorShape([]), 'sentence': TensorShape([])}

# Output types of one entry:
   {'idx': tf.int32, 'label': tf.int64, 'sentence': tf.string}

# Output typesof one entry:
   {'idx': <class 'tensorflow.python.framework.ops.Tensor'>, 'label': <class 'tensorflow.python.framework.ops.Tensor'>, 'sentence': <class 'tensorflow.python.framework.ops.Tensor'>}
 

# Shape of the data:

   (67349,)
   ---> 67349 entries
   ---> 1 dim
        dict structure
           dim: 3
           [idx       / label     / sentence ]
           [()        / ()        / ()       ]
           [int32     / int64     / bytes    ]


# Examples of data:
{'idx': 16399,
 'label': 0,
 'sentence': b'for the uninitiated plays better on video with the sound '}
{'idx': 1680,
 'label': 0,
 'sentence': b'like a g

## Define parameters of the model

In [15]:
# models
#MODELS = [(TFBertModel,     BertTokenizer,       'bert-base-multilingual-uncased'),
#          (OpenAIGPTModel,  OpenAIGPTTokenizer,  'openai-gpt'),
#          (GPT2Model,       GPT2Tokenizer,       'gpt2'),
#          (CTRLModel,       CTRLTokenizer,       'ctrl'),
#          (TransfoXLModel,  TransfoXLTokenizer,  'transfo-xl-wt103'),
#          (XLNetModel,      XLNetTokenizer,      'xlnet-base-cased'),
#          (XLMModel,        XLMTokenizer,        'xlm-mlm-enfr-1024'),
#          (DistilBertModel, DistilBertTokenizer, 'distilbert-base-cased'),
#          (RobertaModel,    RobertaTokenizer,    'roberta-base'),
#          (XLMRobertaModel, XLMRobertaTokenizer, 'xlm-roberta-base'),
#         ]
MODELS = [(TFBertModel,         BertTokenizer,       'bert-base-multilingual-uncased'),
          (TFXLMRobertaModel,   XLMRobertaTokenizer, 'jplu/tf-xlm-roberta-base')]
model_index = 1 #XML Roberta 
model_class        = MODELS[1][0] # i.e TFBertModel
tokenizer_class    = MODELS[1][1] # i.e BertTokenizer
pretrained_weights = MODELS[1][2] #'i.e bert-base-multilingual-uncased'

# Maxium length, becarefull BERT max length is 512!
MAX_LENGTH = 128

# define parameters
BATCH_SIZE_TRAIN = 32
BATCH_SIZE_TEST = 32
BATCH_SIZE_VALID = 64
EPOCH = 2

# extract parameters
size_train_dataset = info.splits['train'].num_examples
size_test_dataset = info.splits['test'].num_examples
size_valid_dataset = info.splits['validation'].num_examples
number_label = info.features["label"].num_classes

# computer parameter
STEP_EPOCH_TRAIN = math.ceil(size_train_dataset/BATCH_SIZE_TRAIN)
STEP_EPOCH_TEST = math.ceil(size_test_dataset/BATCH_SIZE_TEST)
STEP_EPOCH_VALID = math.ceil(size_test_dataset/BATCH_SIZE_VALID)


print('Dataset size:          {:6}/{:6}/{:6}'.format(size_train_dataset, size_test_dataset, size_valid_dataset))
print('Batch size:            {:6}/{:6}/{:6}'.format(BATCH_SIZE_TRAIN, BATCH_SIZE_TEST, BATCH_SIZE_VALID))
print('Step per epoch:        {:6}/{:6}/{:6}'.format(STEP_EPOCH_TRAIN, STEP_EPOCH_TEST, STEP_EPOCH_VALID))
print('Total number of batch: {:6}/{:6}/{:6}'.format(STEP_EPOCH_TRAIN*(EPOCH+1), STEP_EPOCH_TEST*(EPOCH+1), STEP_EPOCH_VALID*(EPOCH+1)))

Dataset size:           67349/  1821/   872
Batch size:                32/    32/    64
Step per epoch:          2105/    57/    29
Total number of batch:   6315/   171/    87


## Tokenizer and prepare data for BERT

In [16]:
# Define the checkpoint directory to store the checkpoints
pretrained_model_dir=savemodel_dir+'/pretrained_model/'+pretrained_weights
os.makedirs(pretrained_model_dir, exist_ok=True)

In [17]:
# Load tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights, cache_dir=pretrained_model_dir)

In [18]:
# recap of input dataset
print(data['train'])
print(tf.data.experimental.cardinality(data['train']))
print(tf.data.experimental.cardinality(data['test']))
print(tf.data.experimental.cardinality(data['validation']))
# super slow since looping over all data
#print(len(list(data['train'])))

<DatasetV1Adapter shapes: {idx: (), label: (), sentence: ()}, types: {idx: tf.int32, label: tf.int64, sentence: tf.string}>
tf.Tensor(-2, shape=(), dtype=int64)
tf.Tensor(-2, shape=(), dtype=int64)
tf.Tensor(-2, shape=(), dtype=int64)


In [19]:
# Prepare data for BERT
train_dataset = glue_convert_examples_to_features(data['train'], 
                                                  tokenizer, 
                                                  max_length=MAX_LENGTH, 
                                                  task='sst-2')
test_dataset = glue_convert_examples_to_features(data['test'], 
                                                  tokenizer, 
                                                  max_length=MAX_LENGTH, 
                                                  task='sst-2')
valid_dataset = glue_convert_examples_to_features(data['validation'], 
                                                  tokenizer, 
                                                  max_length=MAX_LENGTH, 
                                                  task='sst-2')

In [20]:
# recap of pre processing dataset
print(train_dataset)
print(tf.data.experimental.cardinality(train_dataset))
print(tf.data.experimental.cardinality(test_dataset))
print(tf.data.experimental.cardinality(valid_dataset))
# super slow since looping over all data
print(len(list(train_dataset)))

<FlatMapDataset shapes: ({input_ids: (None,), attention_mask: (None,), token_type_ids: (None,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32, token_type_ids: tf.int32}, tf.int64)>
tf.Tensor(-2, shape=(), dtype=int64)
tf.Tensor(-2, shape=(), dtype=int64)
tf.Tensor(-2, shape=(), dtype=int64)
67349


In [21]:
# set shuffle and batch size
train_dataset = train_dataset.shuffle(100).batch(BATCH_SIZE_TRAIN).repeat(EPOCH+1)
test_dataset = test_dataset.shuffle(100).batch(BATCH_SIZE_TEST).repeat(EPOCH+1)
valid_dataset = valid_dataset.batch(BATCH_SIZE_VALID) #.repeat(EPOCH+1)

## Check the final data

In [22]:
pp.print_info_data(train_dataset,print_example=False)

# Structure of the data:

   <RepeatDataset shapes: ({input_ids: (None, None), attention_mask: (None, None), token_type_ids: (None, None)}, (None,)), types: ({input_ids: tf.int32, attention_mask: tf.int32, token_type_ids: tf.int32}, tf.int64)>

# Output shape of one entry:
   ({'input_ids': TensorShape([None, None]), 'attention_mask': TensorShape([None, None]), 'token_type_ids': TensorShape([None, None])}, TensorShape([None]))

# Output types of one entry:
   ({'input_ids': tf.int32, 'attention_mask': tf.int32, 'token_type_ids': tf.int32}, tf.int64)

# Output typesof one entry:
   ({'input_ids': <class 'tensorflow.python.framework.ops.Tensor'>, 'attention_mask': <class 'tensorflow.python.framework.ops.Tensor'>, 'token_type_ids': <class 'tensorflow.python.framework.ops.Tensor'>}, <class 'tensorflow.python.framework.ops.Tensor'>)
 

# Shape of the data:

   (6315, 2)
   ---> 6315 batches
   ---> 2 dim
        label
           shape: (32,)
        dict structure
           dim: 3
        

In [23]:
pp.print_detail_tokeniser(train_dataset, tokenizer)

 input_ids     ---->    attention_mask    token_type_ids    modified text                 

         0     ---->           1                 1          <s>                           
    111767     ---->           1                 1          ``                            
     16195     ---->           1                 1          spi                           
       820     ---->           1                 1          der                           
         9     ---->           1                 1          -                             
       669     ---->           1                 1          man                           
      5106     ---->           1                 1          ''                            
     68782     ---->           1                 1          certainly                     
         2     ---->           1                 1          </s>                          
         0     ---->           0                 0          <s>                          

## Building a classification model

### Use TFBertForSequenceClassification

In [24]:
# Define some parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
# Gradient clipping in the optimizer (by setting clipnorm or clipvalue) is currently unsupported when using a distribution strategy
# clipnorm=1.0

# loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Uses the tf.distribute.MirroredStrategy, which does in-graph replication with synchronous training on many GPUs on one machine
strategy_model_1 = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy_model_1.num_replicas_in_sync))

# create and compile the Keras model in the context of strategy.scope
with strategy_model_1.scope():
    # metric
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    
    # model
    model_1 = TFBertForSequenceClassification.from_pretrained(pretrained_weights,num_labels=number_label, cache_dir=pretrained_model_dir)
    #model.layers[-1].activation = tf.keras.activations.softmax
    model_1._name='tf_bert_classification'
    model_1.compile(optimizer=optimizer,
                    loss=loss, 
                    metrics=[metric])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Number of devices: 1


In [25]:
model_1.summary()

Model: "tf_bert_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  278043648 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 278,045,186
Trainable params: 278,045,186
Non-trainable params: 0
_________________________________________________________________


### Building a custom classification model 

In [46]:
def custom_keras_model(number_classes, bert_model, model_class):

    # create model
    input_layer = tf.keras.Input(shape = (128,), dtype='int64')    
    #bert_ini = TFBertModel.from_pretrained(bert_model, cache_dir=pretrained_model_dir) (input_layer)
    bert_ini = model_class.from_pretrained(bert_model, cache_dir=pretrained_model_dir) (input_layer)
    # This is because in a bert pretraining progress, there are two tasks: 
    # masked token prediction and next sentence predition . 
    # The first needs hidden state of each tokens ( shape: [batch_size, sequence_length, hidden_size]) 
    # the second needs the embedding of the whole sequence (shape : [batch_size, hidden_size] ) .
    bert = bert_ini[1]    
    dropout = tf.keras.layers.Dropout(0.1)(bert)
    flat = tf.keras.layers.Flatten()(dropout)
    classifier = tf.keras.layers.Dense(units=number_classes )(flat) # activation='softmax'               
    
    model = tf.keras.Model(inputs=input_layer, outputs=classifier, name='custom_tf_bert_classification')

    return model, bert_ini

In [47]:
# Define some parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
# Gradient clipping in the optimizer (by setting clipnorm or clipvalue) is currently unsupported when using a distribution strategy
# clipnorm=1.0

# loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Uses the tf.distribute.MirroredStrategy, which does in-graph replication with synchronous training on many GPUs on one machine
strategy_model_2 = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy_model_1.num_replicas_in_sync))

# create and compile the Keras model in the context of strategy.scope
with strategy_model_2.scope():
    # metric
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    
    # model
    model_2, bert_ini = custom_keras_model(number_label, pretrained_weights, model_class)
    model_2.compile(optimizer=optimizer,
                    loss=loss, 
                    metrics=[metric])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Number of devices: 1


In [28]:
bert_ini[0]

<tf.Tensor 'tfxlm_roberta_model/Identity:0' shape=(None, 128, 768) dtype=float32>

In [29]:
bert_ini[1]

<tf.Tensor 'tfxlm_roberta_model/Identity_1:0' shape=(None, 768) dtype=float32>

In [30]:
model_2.summary()

Model: "custom_tf_bert_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
tfxlm_roberta_model (TFXLMRo ((None, 128, 768), (None, 278043648 
_________________________________________________________________
dropout_76 (Dropout)         (None, 768)               0         
_________________________________________________________________
flatten (Flatten)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 1538      
Total params: 278,045,186
Trainable params: 278,045,186
Non-trainable params: 0
_________________________________________________________________


## Select the model for training

In [31]:
model=model_1
print('model\'s name: {}'.format(model.name))

model's name: tf_bert_classification


## Setup of the Callbacks

### TensorBoard

In [32]:
# checking existing folders
for i in os.listdir(tensorboard_dir):
    if os.path.isdir(tensorboard_dir+'/'+i):
        print(i)

20200409-114911


In [33]:
# clean old TensorBoard directory 
for i in os.listdir(tensorboard_dir):
        if os.path.isdir(tensorboard_dir+'/'+i):
            print('remiving:', i)
            shutil.rmtree(tensorboard_dir+'/'+i, ignore_errors=False)

remiving: 20200409-114911


In [34]:
# create a new directory
log_dir=tensorboard_dir+'/'+datetime.now().strftime("%Y%m%d-%H%M%S")
os.mkdir(log_dir)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
                                                      histogram_freq=1, 
                                                      embeddings_freq=1,
                                                      write_graph=True,
                                                      update_freq='batch',
                                                      profile_batch=2)

### Model Checkpoints

In [35]:
# Define the checkpoint directory to store the checkpoints
checkpoint_dir=savemodel_dir+'/checkpoint_model/'+model.name
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch:02d}') 
#checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch:02d}-{accuracy:.4f}')

In [36]:
# clean old checkpoints
for i in os.listdir(checkpoint_dir):
        if os.path.exists(checkpoint_dir+'/'+i):
            os.remove(checkpoint_dir+'/'+i)

In [37]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                                         verbose=1,
                                                         save_weights_only=True)

#checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
#                                                         monitor='accuracy',
#                                                         save_best_only=True,
#                                                         mode='max',
#                                                         save_freq=2,
#                                                         period=5,
#                                                         verbose=1,
#                                                         save_weights_only=True)

### Decaying learning rate

In [38]:
decay_callback = tf.keras.callbacks.LearningRateScheduler(mu.decay)

### Checks callbacks

In [39]:
list_callback = [tensorboard_callback, checkpoint_callback, decay_callback]
for cb in list_callback:
    if type(cb).__name__=='tuple':
        print(cb[0].__class__.__name__, 'need to unpack this tuple by adding *')

Maybe some callback need to unpack this tuple by adding *

## TensorBoard

In [40]:
%load_ext tensorboard
#%reload_ext tensorboard
%tensorboard  --logdir   {log_dir}

## Training

In [41]:
# input format depending of the type of model
def data_feature_extraction(data, name):
    if name=='custom_tf_bert_classification':
        print('custom model: {}'.format(name))
        return data.map(pp.feature_selection)
    elif name=='tf_bert_classification':
        print('standard model: {}'.format(name))
        return data
    else:
        print('!!! non defined model !!!!')
        return data

In [42]:
# Clean up existing  weigts and values
tf.keras.backend.clear_session()

In [43]:
# time the function
start_time = time.time()

# making the transformation here since insude model.fit it create a lot of warnings
data_train = data_feature_extraction(train_dataset, model.name)
data_val = data_feature_extraction(valid_dataset, model.name)

# callback to creaate  history per step (not per epoch)
histories_per_step = mu.History_per_step(data_val, 10)


# train the model
history = model.fit(data_train, 
                    epochs=1, 
                    steps_per_epoch=10, #STEP_EPOCH_TRAIN,
                    validation_data=data_val,
                    validation_steps=3,
                    callbacks=[tensorboard_callback,
                               checkpoint_callback,
                               histories_per_step])

# print execution time
elapsed_time_secs = time.time() - start_time
print('\nexecution time: {}'.format(timedelta(seconds=round(elapsed_time_secs))))

standard model: tf_bert_classification
standard model: tf_bert_classification
Train for 10 steps, validate for 3 steps

 training set -> batch:1 loss:0.6908279061317444 and acc: 0.5
 1/10 [==>...........................] - ETA: 6:22 - loss: 0.6908 - accuracy: 0.5000
 training set -> batch:2 loss:0.6876528263092041 and acc: 0.578125
 2/10 [=====>........................] - ETA: 4:27 - loss: 0.6892 - accuracy: 0.5781
 training set -> batch:3 loss:0.6780118942260742 and acc: 0.5520833134651184
 3/10 [========>.....................] - ETA: 3:24 - loss: 0.6855 - accuracy: 0.5521
 training set -> batch:4 loss:0.6834325790405273 and acc: 0.5625
 4/10 [===========>..................] - ETA: 2:41 - loss: 0.6850 - accuracy: 0.5625
 training set -> batch:5 loss:0.8453351259231567 and acc: 0.5249999761581421
 5/10 [==============>...............] - ETA: 2:10 - loss: 0.7171 - accuracy: 0.5250
 training set -> batch:6 loss:0.7371960282325745 and acc: 0.515625
 6/10 [=================>............] -

In [61]:
mu.load_data_tensorboard(log_dir+'/train')

{'batch_accuracy': (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
  array([0.5       , 0.578125  , 0.55208331, 0.5625    , 0.52499998,
         0.515625  , 0.5223214 , 0.5390625 , 0.5451389 , 0.55312502])),
 'batch_loss': (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
  array([0.69082791, 0.68765283, 0.67801189, 0.68343258, 0.84533513,
         0.73719603, 0.72645992, 0.63484412, 0.68087548, 0.66873586])),
 'epoch_accuracy': (array([0]), array([0.55312502])),
 'epoch_loss': (array([0]), array([0.70333719]))}

In [44]:
stop here !

SyntaxError: invalid syntax (<ipython-input-44-48709baa0273>, line 1)

## Save the model and histories

### Save the model

In [ ]:
savemodel_path = os.path.join(savemodel_dir, 'saved_model')
os.makedirs(savemodel_path, exist_ok=True)

In [ ]:
model.save(os.path.join(savemodel_path,model.name))

In [ ]:
# check the saved model
print('Model: {}'.format(model.name))
for i in os.listdir(os.path.join(savemodel_path,model.name)):
        print(' ',i)
            

The **variables** directory contains a standard training checkpoint (see the guide to training checkpoints).   

The **assets** directory contains files used by the TensorFlow graph, for example text files used to initialize vocabulary tables.  

The **saved_model.pb** file stores the actual TensorFlow program, or model, and a set of named signatures, each identifying a function that accepts tensor inputs and produces tensor outputs. SavedModels may contain multiple variants of the model (multiple v1.MetaGraphDefs, identified with the --tag_set flag to saved_model_cli), but this is rare. APIs which create multiple variants of a model include.

### Save the histories

In [ ]:
with open(savemodel_path+'/'+model.name+'/history', 'wb') as file:
    model_history= mu.History_trained_model(history.history, history.epoch, history.params)
    pickle.dump(model_history, file, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(savemodel_path+'/'+model.name+'/history_per_step', 'wb') as file:
    model_history_per_step= mu.History_per_steps_trained_model(histories_per_step.steps, 
                                                               histories_per_step.losses, 
                                                               histories_per_step.accuracies, 
                                                               histories_per_step.val_steps, 
                                                               histories_per_step.val_losses, 
                                                               histories_per_step.val_accuracies)
    pickle.dump(model_history_per_step, file, pickle.HIGHEST_PROTOCOL)

## Metrics during training

### Visualization

In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]

In [ ]:
mm.plot_acc_loss(steps_loss_train=histories_per_step.steps, loss_train=histories_per_step.losses,
                 steps_acc_train=histories_per_step.steps, accuracy_train=histories_per_step.accuracies,
                 steps_loss_eval=histories_per_step.val_steps, loss_eval=histories_per_step.val_losses,
                 steps_acc_eval=histories_per_step.val_steps, accuracy_eval=histories_per_step.val_accuracies)

### Metadata

In [ ]:
print(model.metrics)
print(model.metrics_names)

In [ ]:
history.epoch

In [ ]:
history.params

In [ ]:
history.history.keys()

In [ ]:
# dir(history)

### Exploration of the model's structure

In [ ]:
tf.keras.utils.plot_model(model,
                          'model.png',
                          show_shapes=True)

In [ ]:
model.inputs

In [ ]:
model.outputs

In [ ]:
model.layers

In [ ]:
# _inbound_nodes and inbound_nodes give the same !
# to see method available: dir(model.layers[2])
for layer in model.layers:
    print(layer.name, layer._inbound_nodes, layer._outbound_nodes)

## Model's evaluation and validation

### Making prediction

In [ ]:
# get probablility for each classes
if model.name=='custom_tf_bert_classification':
        print('custom model: {}'.format(model.name))
        y_pred = tf.nn.softmax(model.predict(valid_dataset))
elif model.name=='tf_bert_classification':
        print('standard model: {}'.format(model.name))
        y_pred = tf.squeeze(tf.nn.softmax(model.predict(valid_dataset)))

In [ ]:
y_pred.shape

In [ ]:
# get predicted classes
y_pred_argmax = tf.math.argmax(y_pred, axis=1)        

In [ ]:
(y_pred_argmax).shape

### Extracting true classes

In [ ]:
# extracting and flatten true classes
y_true_tf=valid_dataset.map(pp.label_extraction).flat_map(lambda x: valid_dataset.from_tensor_slices(x))

In [ ]:
y_true=list(y_true_tf.as_numpy_iterator())

In [ ]:
len(y_true), len(y_pred_argmax)

### Model performanes

In [ ]:
print(classification_report(y_true, y_pred_argmax, target_names=info.features["label"].names))

In [ ]:
mm.print_metrics(y_true, y_pred_argmax, mode='')

### Confusion matrix

In [ ]:
plt.rcParams['figure.figsize'] = [10, 7]

In [ ]:
mm.plot_confusion_matrix(confusion_matrix(y_true, y_pred_argmax), info.features["label"].names)

In [ ]:
mm.plot_confusion_matrix(confusion_matrix(y_true, y_pred_argmax), info.features["label"].names, normalize=True)

### ROC curves

In [ ]:
mm.roc_curves(to_categorical(y_true), y_pred.numpy(), info.features["label"].names)

## Model's inference

In [ ]:
# getting a random example from the test data
for i in data['test'].shuffle(100).take(1):
    print('sentence: {}\nlabel:    {}'.format(i['sentence'].numpy().decode(), info.features["label"].names[i['label'].numpy()]))

In [ ]:
# translate some example in some languages
text = ["Zhang Yimou livre des personnages chaleureux et authentiques qui mentent non par malhonnêteté, mais parce qu'ils croient sincèrement que c'est la seule façon d'apporter le bonheur à leurs proches."]

In [ ]:
def made_prediction(text):
    print('example of input:\n\n{}\n \nlength:{}\n'.format(text[0], len(text)))
    #print('text:{} length:{}\n'.format(text, len(text)))
    # get probablility for each classes
    tokens=tokenizer.batch_encode_plus(text, return_tensors="tf",  pad_to_max_length=True)
    digits=model.predict(tokens)
    if model.name=='custom_tf_bert_classification':
        print('custom model: {}'.format(model.name))
        y_single_pred = tf.nn.softmax(digits)
    elif model.name=='tf_bert_classification':
        print('standard model: {}'.format(model.name))
        temp=tf.nn.softmax(digits)
        y_single_pred = tf.squeeze(temp)
        return y_single_pred.numpy()

In [ ]:
y_single_pred=made_prediction(text)

In [ ]:
y_single_pred

In [ ]:
info.features["label"].names[tf.math.argmax(y_single_pred).numpy()]

### Model interpretation

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=info.features["label"].names)

In [ ]:
# will do 5'000 permutations/selections of the original sentence and made prediction
exp = explainer.explain_instance(text[0], made_prediction, num_features=6)

In [ ]:
exp.as_list()

In [ ]:
plt.style.use('dark_background')
fig = exp.as_pyplot_figure()

In [ ]:
exp.show_in_notebook(text=True)

In [ ]:
exp.save_to_file('ex.html')

In [ ]:
from IPython.display import IFrame    
IFrame(src="ex.html", width=900, height=350)

In [ ]:
from IPython.core.display import display, HTML
display(HTML('<iframe src=ex.html width=900 style="background: #FFFFFF;" height=350></iframe>'))

### Copy the model on GCP

The variables directory contains a standard training checkpoint (see the guide to training checkpoints).  
The assets directory contains files used by the TensorFlow graph, for example text files used to initialize vocabulary tables.
The saved_model.pb file stores the actual TensorFlow program, or model, and a set of named signatures, each identifying a function that accepts tensor inputs and produces tensor outputs.

SavedModels may contain multiple variants of the model (multiple v1.MetaGraphDefs, identified with the --tag_set flag to saved_model_cli), but this is rare. APIs which create multiple variants of a model include 

In [ ]:
def copy_local_directory_to_gcs(local_path, bucket, gcs_path):
    """Recursively copy a directory of files to GCS.

    local_path should be a directory and not have a trailing slash.
    """
    assert os.path.isdir(local_path)
    for root, dirs, files in os.walk(local_path):
        for name in files:
            local_file = os.path.join(root, name)
            remote_path = os.path.join(gcs_path, local_file[1 + len(local_path) :])
            print(remote_path)
            blob = bucket.blob(remote_path)
            blob.upload_from_filename(local_file)
            print('copy of the file on gs:// done !')

In [ ]:
storage_client = storage.Client()
bucket = storage_client.get_bucket(os.environ['BUCKET_NAME'])

In [ ]:
source_bucket_name = savemodel_path+'/'+model.name
destination_bucket_name = 'saved_model/tf_bert_classification'

In [ ]:
copy_local_directory_to_gcs(source_bucket_name, bucket, destination_bucket_name)

In [ ]:
for root, dirs, files in os.walk(source_bucket_name):
    for name in files:
        print(os.path.join(root, name))

### Online Predictions

#### Defined a name for our model on GCP and create it

In [ ]:
# defining the name of the model for online prediction
os.environ['MODEL_NAME']=model.name+'_test'

In [ ]:
# Normal VM has a model size of 500 MB for more you need to use a specific n1-standard-2 VM (2 GB) for online prediction. It is only available in us-central1.
os.environ['REGION_PRED']='us-central1'

In [ ]:
%%bash
gcloud ai-platform models create $MODEL_NAME \
       --regions=$REGION_PRED \
       --enable-logging

#### Defined parameters and upload our models

In [ ]:
# define python and run time version
os.environ['RUNTIME_VERSION'] = '2.1'
os.environ['PYTHON_VERSION'] = '3.7'
os.environ['MODEL_BINARIES'] = 'gs://'+os.environ['BUCKET_NAME']+'/saved_model/'+model.name
os.environ['MODEL_VERSION'] = 'v1'

In [ ]:
%%bash
gcloud beta ai-platform versions create $MODEL_VERSION \
       --model $MODEL_NAME \
       --origin $MODEL_BINARIES \
       --runtime-version $RUNTIME_VERSION \
       --python-version $PYTHON_VERSION \
       --machine-type n1-standard-2 \
       --description "This is sentiment classifier using BERT and fine tune on SST-2 for test"

#### Check models already deployed

In [ ]:
%%bash
gcloud ai-platform models list

#### Prepare data for online prediction

In [ ]:
# example of format:

```--json-request  
  {  
    "instances": [  
      {"x": [1, 2], "y": [3, 4]},  
      {"x": [-1, -2], "y": [-3, -4]}  
    ]  
  }  ```
  
```--json-instances  
  {"images": [0.0, …, 0.1], "key": 3}  
  {"images": [0.0, …, 0.1], "key": 2}  ```

In [ ]:
np_array = np.array(list(valid_dataset.unbatch().as_numpy_iterator()))

In [ ]:
np_array.shape

In [ ]:
import codecs
import json
json_file = "input_predict_gcloud.json" 

with codecs.open(json_file, 'w', encoding='utf-8') as f:
    for el in np_array[0:2]:
        #print(el[0]['input_ids'])
        #print(el[0]['input_ids'].shape, el[0]['input_ids'].dtype)
        #print(el[0]['attention_mask'].shape, el[0]['attention_mask'].dtype)
        #print(el[0]['token_type_ids'].shape, el[0]['token_type_ids'].dtype)
        instance={'input_ids': el[0]['input_ids'].tolist(), 'attention_mask': el[0]['attention_mask'].tolist(), 'token_type_ids': el[0]['token_type_ids'].tolist()}
        json.dump(instance, f , sort_keys=True)
        f.write("\n")

In [ ]:
# how to look at these data ?

In [ ]:
with open('input_predict_gcloud.json') as json_file:
    for i in json_file:
        # convert string to dictionary
        json_data = eval(i)
        print(json_data.keys())
        print(json_data)

#### Make local prediction to test the model and input data

In [ ]:
os.environ['CLOUDSDK_PYTHON']='/Users/tarrade/anaconda-release/conda-env/env_gcp_sdk/bin/python'
os.environ['CLOUDSDK_GSUTIL_PYTHON']='/Users/tarrade/anaconda-release/conda-env/env_gcp_sdk/bin/python'

In [ ]:
os.environ['MODEL_LOCAL']=savemodel_path+'/'+model.name

In [ ]:
%%bash
saved_model_cli show --dir $MODEL_LOCAL --tag_set serve 

In [ ]:
%%bash
saved_model_cli show --dir $MODEL_LOCAL --tag_set serve --signature_def serving_default

In [ ]:
%%bash
gcloud ai-platform local predict \
       --signature-name serving_default \
       --model-dir $MODEL_LOCAL \
       --json-instances input_predict_gcloud.json \
       --verbosity info \
       #--log-http 

#### Make online prediction

In [ ]:
%%bash
gcloud ai-platform predict \
       --model $MODEL_NAME \
       --version $MODEL_VERSION \
       --json-instances input_predict_gcloud.json\
       --verbosity info \
       #--log-http

# Data analysis

In [ ]:
def extraction(feature):
    print('feature:',feature['sentence'])
    
    return feature['sentence'], feature['label']

In [ ]:
for i in a:
    print(i[0].numpy())
    break

In [ ]:
np_array = np.array(list(data['train'].map(extraction).as_numpy_iterator()))

In [ ]:
np_array.shape

In [ ]:
import pandas as pd
df = pd.DataFrame(data=np_array, columns=['sentence', 'label'])

In [ ]:
df['text']=df['sentence'].apply(lambda x: x.decode("utf-8"))

In [ ]:
df['label']=df['label'].astype('int32')

In [ ]:
df.columns

In [ ]:
df.pop('sentence')

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
len(df)

In [ ]:
df.dtypes

In [ ]:
df['label'].value_counts(dropna=False)

In [ ]:
df['word_count'] = df['text'].apply(lambda txt: len(txt.split(' ')))

In [ ]:
df.head()

In [ ]:
import seaborn as sns
q=0.95
x = df['word_count']
sns.distplot(x, hist=True, rug=True);
print('Minimum word count required to include all words in {}% of the reviews: {}'.format(q*100, x.quantile(q)))

In [ ]:
df['word_count'].describe()


In [ ]:
kwargs = {'cumulative': True}
sns.distplot(x, hist_kws=kwargs);

In [ ]:
tot=len(df)
for size in [2, 5, 10, 20, 27]:
    nb=len(df[df['word_count'] <size])
    print('thresold: {:10d}  amount of data selected: {:10d} percentage of the data selected: {:.2f} %'.format(size, nb , nb/tot*100))